<a href="https://colab.research.google.com/github/jmlDC/MediaBias-Thesis22-23/blob/Modeling/BERT_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installs

In [1]:
!pip install transformers datasets evaluate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 6.0 MB/s eta 0:00:00


In [3]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate
!pip install git+https://github.com/huggingface/accelerate
!pip uninstall -y transformers
!pip install transformers==4.28.0

Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-0n8dcsda
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-0n8dcsda
  Resolved https://github.com/huggingface/accelerate to commit 70920895e80f78d96d8f91e0beeb3ebdb8e5e5d6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproj

## Token hugginng face

In [4]:
from huggingface_hub import notebook_login

notebook_login()

## Dataset

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

dir  = "/content/gdrive/MyDrive/THESIS-MS/Git-Thesis22-23/"

Mounted at /content/gdrive


In [6]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv(f'{dir}Official/MFC_prepared.csv', usecols=["code_frames", "annotations"], header=0)
df.code_frames = df.code_frames.astype(int)

label_dict = {}
for x in range(15):
    label_dict[x+1] = x

label_dict
df['label'] = df.code_frames.replace(label_dict)
df

df = pd.DataFrame(df)
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.2)

# dataset = dataset.rename_column("code_frames", "label")
dataset = dataset.rename_column("annotations", "text")

dataset

DatasetDict({
    train: Dataset({
        features: ['code_frames', 'text', 'label'],
        num_rows: 37439
    })
    test: Dataset({
        features: ['code_frames', 'text', 'label'],
        num_rows: 9360
    })
})

## Prep 

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased",
                                          do_lower_case=False)


In [8]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, 
    max_length=128)

In [9]:
mfc_tokenized = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/37439 [00:00<?, ? examples/s]

Map:   0%|          | 0/9360 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
import evaluate

accuracy = evaluate.load("accuracy")

In [12]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [16]:
id2label, label2id = {},{}
for x in range(15):
    id2label[x] = x
        # {0: "NEGATIVE", 1: "POSITIVE"}
    label2id[x] = x
        # {"NEGATIVE": 0, "POSITIVE": 1}

In [17]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased", num_labels=15, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

## Training

In [18]:
training_args = TrainingArguments(
    output_dir="Bert-trial",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mfc_tokenized["train"],
    eval_dataset=mfc_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/content/Bert-trial is already a clone of https://huggingface.co/jmLuis/Bert-trial. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.056600,0.934590,0.716453
2,0.774700,0.878050,0.732372
3,0.539900,0.896228,0.740705
4,0.383200,0.974471,0.744444
5,0.276500,1.087449,0.739637
6,0.200400,1.166801,0.742521
7,0.158200,1.229310,0.746261
8,0.130000,1.311914,0.744338
9,0.107100,1.349917,0.748504
10,0.090800,1.373488,0.747329


TrainOutput(global_step=11700, training_loss=0.38943232919415854, metrics={'train_runtime': 8952.2159, 'train_samples_per_second': 41.821, 'train_steps_per_second': 1.307, 'total_flos': 1.7475327949322484e+16, 'train_loss': 0.38943232919415854, 'epoch': 10.0})

In [20]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/May23_06-27-47_b0d428ad89d4/events.out.tfevents.1684823269.b0d428ad89d4.195.0:   0%|         …

To https://huggingface.co/jmLuis/Bert-trial
   e6654cd..3bf1cfe  main -> main

   e6654cd..3bf1cfe  main -> main

To https://huggingface.co/jmLuis/Bert-trial
   3bf1cfe..57ac3af  main -> main

   3bf1cfe..57ac3af  main -> main



'https://huggingface.co/jmLuis/Bert-trial/commit/3bf1cfedf8342de533dac196663939f14f2e4824'

# Inference

In [62]:
from transformers import AutoModelForSequenceClassification
import torch
from transformers import AutoTokenizer


In [91]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."
text = 'Immigrants without HOPE need help entering college'

In [92]:
tokenizer = AutoTokenizer.from_pretrained("jmLuis/Bert-trial")
inputs = tokenizer(text, return_tensors="pt")

In [93]:
model = AutoModelForSequenceClassification.from_pretrained("jmLuis/Bert-trial")
with torch.no_grad():
    logits = model(**inputs).logits

In [95]:
predicted_class_id = logits.argmax().item()
out = model.config.id2label[predicted_class_id]
out+1

10

In [96]:
def infer_bert(row):
    tokenizer = AutoTokenizer.from_pretrained("jmLuis/Bert-trial")
    inputs = tokenizer(row['Text'], return_tensors="pt")
    model = AutoModelForSequenceClassification.from_pretrained("jmLuis/Bert-trial")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    out = model.config.id2label[predicted_class_id]
    return int(out)+1  #adding onw because original label is not zero index, however training is.

## applying in curated dataset

In [97]:
curated_df = pd.read_csv(f'{dir}Official/curated_Dataset.csv', usecols=["Annotate", "Text"], header=0)

curated_df

,Annotate,Text
0,NaN,"DOH logs 1,920 new COVID-19 cases, highest dai..."
1,NaN,"The Philippines on Sunday registered 1,920 ne..."
2,NaN,"According to GMA Integrated News Research, th..."
3,NaN,"It is the fourth straight day of more than 1,..."
4,NaN,Data from the Department of Health's COVID-19...
...,...,...
132092,NaN,"""The Philippines and China also discussed new ..."
132093,NaN,"On Thursday, Lazaro started chairing with Chin..."
132094,NaN,"The conduct of the BCM, a confidence-building ..."
132095,NaN,"In an earlier pronouncement, Lazaro asserted t..."


In [98]:
curated_df.shape

(132097, 2)

In [ ]:
for i, row in curated_df.iterrows():
    curated_df.at[i,'Annotate'] = infer_bert(row)

In [ ]:
curated_df.to_csv(f'{dir}Bert-curated_dataset.csv', index=False)

In [88]:
curated_df

,Annotate,Text
0,2.0,"DOH logs 1,920 new COVID-19 cases, highest dai..."
1,14.0,"The Philippines on Sunday registered 1,920 ne..."
2,11.0,"According to GMA Integrated News Research, th..."
3,9.0,"It is the fourth straight day of more than 1,..."
4,9.0,Data from the Department of Health's COVID-19...
...,...,...
132092,NaN,"""The Philippines and China also discussed new ..."
132093,NaN,"On Thursday, Lazaro started chairing with Chin..."
132094,NaN,"The conduct of the BCM, a confidence-building ..."
132095,NaN,"In an earlier pronouncement, Lazaro asserted t..."
